# Plots

In [2]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2
import os
import pandas as pd
import numpy as np
import glob

## Plot different classes in different colors on the same image

In [2]:
# select slice and mode 
# slice ranging from 400th to 800th, 401 in total
# seg_model: k-means/gmm/dragonfly
# mode: number of classes, either 16, 32, 64, 128 or 4 for dragonfly
slice_num = 400
seg_model = 'k-means'
seg_nd = '3d'
mode = 16

In [3]:
assert (seg_model == 'dragonfly') == (mode == 4)

In [2]:
# data folder path
current_path = os.getcwd()

# dragonfly
def get_dragonfly_images(slice_num):
    s = slice_num - 400 + 1
    s = str(s).zfill(3)
    dragonfly_bassanite = os.path.join(current_path, 'VA10_0050_Bassanite\VA10_0050_Bassanite{}.tiff'.format(s))
    dragonfly_celestite = os.path.join(current_path, 'VA10_0050_Celestite\VA10_0050_Celestite{}.tiff'.format(s))
    dragonfly_gypsum = os.path.join(current_path, 'VA10_0050_Gypsum\VA10_0050_Gypsum{}.tiff'.format(s))
    dragonfly_pore = os.path.join(current_path, 'VA10_0050_Pores\VA10_0050_Pores{}.tiff'.format(s))

    bassanite = cv2.imread(dragonfly_bassanite)
    celestite = cv2.imread(dragonfly_celestite)
    gypsum = cv2.imread(dragonfly_gypsum)
    pore = cv2.imread(dragonfly_pore)

    return bassanite, celestite, gypsum, pore

def get_dragonfly_grayscale(slice_num):
    s = slice_num - 400 + 1
    s = str(s).zfill(3)
    dragonfly_bassanite = os.path.join(current_path, 'VA10_0050_Bassanite\VA10_0050_Bassanite{}.tiff'.format(s))
    dragonfly_celestite = os.path.join(current_path, 'VA10_0050_Celestite\VA10_0050_Celestite{}.tiff'.format(s))
    dragonfly_gypsum = os.path.join(current_path, 'VA10_0050_Gypsum\VA10_0050_Gypsum{}.tiff'.format(s))
    dragonfly_pore = os.path.join(current_path, 'VA10_0050_Pores\VA10_0050_Pores{}.tiff'.format(s))

    bassanite = cv2.imread(dragonfly_bassanite, cv2.IMREAD_GRAYSCALE) / 255
    celestite = cv2.imread(dragonfly_celestite, cv2.IMREAD_GRAYSCALE) / 255
    gypsum = cv2.imread(dragonfly_gypsum, cv2.IMREAD_GRAYSCALE) / 255
    pore = cv2.imread(dragonfly_pore, cv2.IMREAD_GRAYSCALE) / 255

    return bassanite, celestite, gypsum, pore

# gmm / k-means
def get_seg_image_path(model, nd, mode, slice_num):
   seg_list = glob.glob(os.path.join(current_path, 'new_large_clusters_rec', model,  nd, 'cluster_{}'.format(mode), slice_num))



In [3]:
bassanite, celestite, gypsum, pore = get_dragonfly_images(slice_num)


NameError: name 'slice_num' is not defined

In [4]:
# Color Palette
black = [0,0,0]
white = [255,255,255]
color_1 = [118,42,131]
color_2 = [175,141,195]
color_3 = [231,212,232]
color_4 = [217,240,211]
color_5 = [127,191,123]
color_6 = [27,120,55]

In [5]:
# load related images
#if mode == 4:
bassanite, celestite, gypsum, pore = get_dragonfly_images(471)
bassanite[np.all(bassanite == white, axis=-1)] = color_2
celestite[np.all(celestite == white, axis=-1)] = color_3
gypsum[np.all(gypsum == white, axis=-1)] = color_4
pore[np.all(pore == white, axis=-1)] = color_5

img = bassanite + celestite + gypsum + pore
cv2.imshow('w', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
#cv2.imwrite('dragonfly_visualisation_report_471.png', img)

In [17]:
# test intersection of classes from dragonfly => no overlapping
b_gray, c_gray, g_gray, p_gray = get_dragonfly_grayscale(400)
all_add = b_gray + c_gray + g_gray + p_gray
all_add

array([[0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [0., 0., 1., ..., 1., 1., 1.],
       [0., 0., 1., ..., 1., 1., 1.]])

In [20]:
print((all_add == 0).sum())   # 25492 pixels were not classified as any classes here

25492


In [21]:
25492 / (700*855)   # indeed within the 95% accuaracy range

0.04259314954051796

# labeling segmentation results and combine same class together

pore: 0

gypsum: 1

celestite: 2

bassanite: 3

In [10]:
# for my segmentation results: concate up for 
# since the clusters are gathering because of the same centroids, if there is an empty cluster, then that means in that image, there isn't any pixel that could be considered as that cluster.
# Empty is a signal of no some class, so it is important to keep it as that class. 


"""need to find the average of each cluster, because as z goes larger, gypsum might goes down, pores and bassanie might grow and hence has more dominant data at around 750-800 slices but are tie with gypsum at the beginning.  """

# the following code was for k-means 3d 16

label = [0]*16
label[0] = 1
label[1] = 2
label[2] = 0  # 0/1 weird that presicion of pore was obviously larger but then it drops
label[3] = 0
label[4] = 1
label[5] = 3
label[6] = 0
label[7] = 1
label[8] = 1
label[9] = 3
label[10] = 0
label[11] = 3
label[12] = 0  #0/1  # weird that presicion of pore was obviously larger but then it drops
label[13] = 1  #0/0/3
label[14] = 1
label[15] = 3



In [11]:
seg_model = 'k-means'
seg_nd = '3d'
mode = 16


df = pd.read_csv('{}_{}_{}_f1.csv'.format(seg_model, seg_nd, mode))

In [12]:
same_class = np.array(label*401)  # the res from same group mush have same label

df['same_class'] = same_class
df

,slice,current_cluster,pore_micro_precision,pore_micro_recall,pore_micro_f1,pore_macro_precision,pore_macro_recall,pore_macro_f1,gypsum_micro_precision,gypsum_micro_recall,...,celestite_macro_precision,celestite_macro_recall,celestite_macro_f1,bassanite_micro_precision,bassanite_micro_recall,bassanite_micro_f1,bassanite_macro_precision,bassanite_macro_recall,bassanite_macro_f1,same_class
0,400,0,0.012649,0.029551,0.017715,0.012855,0.030710,0.017283,0.918107,0.162058,...,0.000000,0.000000,0.000000,0.050779,0.036827,0.042692,0.051571,0.036910,0.041451,1
1,400,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2
2,400,2,0.495717,0.185133,0.269586,0.485919,0.192986,0.266431,0.445632,0.012575,...,0.000000,0.000000,0.000000,0.040623,0.004710,0.008441,0.042345,0.005007,0.008756,0
3,400,3,0.773465,0.166946,0.274618,0.756065,0.163197,0.256818,0.129570,0.002113,...,0.000000,0.000000,0.000000,0.089485,0.005996,0.011239,0.082403,0.005678,0.010376,0
4,400,4,0.048145,0.068911,0.056686,0.048326,0.074037,0.054871,0.881532,0.095334,...,0.000000,0.000000,0.000000,0.054871,0.024381,0.033761,0.055902,0.025334,0.033623,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6411,800,11,0.003934,0.002790,0.003265,0.004146,0.002722,0.003128,0.000000,0.000000,...,0.000179,0.005848,0.000346,0.990529,0.268365,0.422312,0.990973,0.267241,0.413764,3
6412,800,12,0.360981,0.117477,0.177265,0.348757,0.115484,0.171463,0.328915,0.009191,...,0.000000,0.000000,0.000000,0.281213,0.034968,0.062202,0.284500,0.034887,0.061250,0
6413,800,13,0.224580,0.054941,0.088285,0.231613,0.055754,0.085039,0.428888,0.009009,...,0.000000,0.000000,0.000000,0.321297,0.030033,0.054932,0.309045,0.030372,0.054245,1
6414,800,14,0.006191,0.012941,0.008375,0.006166,0.014447,0.008237,0.942736,0.169204,...,0.000000,0.000000,0.000000,0.037221,0.029727,0.033055,0.038444,0.029277,0.032186,1


In [1]:
zero = df.loc[df['current_cluster'] == 0]

NameError: name 'df' is not defined

In [2]:
zero.describe()

NameError: name 'zero' is not defined

# Notes

In [ ]:
# 3d and 4d similarity, strong active sense in the middle. Even though the representation of activeness in term of time and space might be different, but they are both presenting the same reaction process. Some other regions at two ends are experiencing the previous stage of the middle regions. 
# Similarly, some later reaction stage would happen at the region at the ends.
# Hence, understanding z axis is important for understanding the axis of time. 



# Image comparison

In [32]:
p1 = os.path.join(os.getcwd(), 'report_images\T0050_0800_4d_cluster_6_2.png')
p2 = os.path.join(os.getcwd(), 'report_images\T0050_0800_4d_cluster_6_1.png')
img1 = ~cv2.imread(p1, 0)
img2 = ~cv2.imread(p2, 0)

both = img1 + img2
flip = ~both
cv2.imwrite('report_images/T0050_0800_4d_cluster_6_1+2.png', flip)

# cv2.imshow('d', both)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


# diff = (img1 != img2).any(axis=2).sum()
# ratio = 1- (diff / (700*855))
# print(diff)
# print(ratio)


True

In [20]:

# both = x+y
# invert_both = ~both
# #cv2.imshow('d', invert_both)
# cv2.imshow('d', x)
# cv2.waitKey(0)

# cv2.destroyAllWindows()
check = x+y


In [21]:
check

array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)

In [27]:
# compare if 6-2 plus 6-1 and 3-2
p3 = os.path.join(os.getcwd(), 'report_images\T0050_0800_4d_cluster_3_2.png')
img3 = cv2.imread(p3, 0)
im3 = img3.flatten()
z = ~im3


In [30]:
dif = (check != z).sum()
dif/(700*855)

0.149593984962406

In [28]:
cv2.imshow('w', x)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Images for Report

In [25]:
file_path = os.path.join(os.getcwd(), 'cropped_0050\VA10_Pc200_Ram25_Pf50_T125_0050_0445.rec.8bit.tif')
img = cv2.imread(file_path, -1) #[353:1053, 282:1137]
cv2.imwrite('report_images/raw_data_to_png/VA10_0050_0445.rec.8bit.png', img)


True

In [7]:
# color_1 = [118,42,131]
# color_2 = [175,141,195]
# color_3 = [231,212,232]
# color_4 = [217,240,211]
# color_5 = [127,191,123]
# color_6 = [27,120,55]
black = [0,0,0]
gray = [150,150,150]
white = [255,255,255]
orange = [92,204,254]
red = [28,26,227]
pink = [185,180,251]
color_2 = [180,218,161]
color_3 = [196,182,65]
color_4 = [204,255,255]
color_5 = [168,94,34]
blue = [184,127,44]

# Dragonfly Visualisation

In [8]:

bassanite, celestite, gypsum, pore = get_dragonfly_images(618)
bassanite[np.all(bassanite == white, axis=-1)] = orange
celestite[np.all(celestite == white, axis=-1)] = blue
gypsum[np.all(gypsum == white, axis=-1)] = gray
pore[np.all(pore == white, axis=-1)] = red

img = bassanite + celestite + gypsum + pore
cv2.imwrite('report_images\dragonfly_visualisation_report_618.png', img)

True

In [22]:
# gmm / k-means
def color_unsupervised_pred_by_slice(manual_label, seg_model, seg_nd, cluster_num, slice_num):
    current_path = os.getcwd()
    seg_path = os.path.join(current_path, 'new_large_clusters_rec', seg_model, seg_nd, 'cluster_{}'.format(cluster_num))
    seg_res_list = glob.glob(os.path.join(seg_path, str(slice_num), '*.png'))
    
    img = 0

    assert len(manual_label) == len(seg_res_list) == cluster_num

    for seg in seg_res_list:
        # our segmentation results has invered color, black [0,0,0] is the interested class
        c = int(os.path.basename(seg[-16:-13]))
        if manual_label[c] == 1:  # pore
            pore = cv2.imread(seg)
            pore[np.all(pore == black, axis=-1)] = red
            pore[np.all(pore == white, axis=-1)] = black
            img += pore
        elif manual_label[c] == 2:  #gypsum
            gypsum = cv2.imread(seg)
            gypsum[np.all(gypsum == black, axis=-1)] = gray
            gypsum[np.all(gypsum == white, axis=-1)] = black
            img += gypsum
        elif manual_label[c] == 3:  #celestite
            celestite = cv2.imread(seg)
            celestite[np.all(celestite == black, axis=-1)] = blue
            celestite[np.all(celestite == white, axis=-1)] = black
            img += celestite
        elif manual_label[c] == 4:  #bassanite
            bassanite = cv2.imread(seg)
            bassanite[np.all(bassanite == black, axis=-1)] = orange
            bassanite[np.all(bassanite == white, axis=-1)] = black
            img += bassanite

    return img

In [23]:
def auto_label_last_201(seg_model, seg_nd, cluster_num, mode, threshold=0.5):
    # cluster_num: the total number of clusters
    assert mode in [1,2,3,4], "Invalid mode: mode should be integer in [1,2,3,4]."
    if mode == 1:
        assert threshold == 0.5, "Mode 1 requires threshold = 0.5."

    csv_file = os.path.join(os.getcwd(), 'evaluation_rec_f1', '{}_{}_{}_f1.csv'.format(seg_model, seg_nd, cluster_num))
    df = pd.read_csv(csv_file, usecols=['slice', 'current_cluster','pore_micro_precision', 'pore_micro_f1', 'gypsum_micro_precision', 'gypsum_micro_f1', 'celestite_micro_precision', 'celestite_micro_f1', 'bassanite_micro_precision', 'bassanite_micro_f1'])
    df =  df.loc[df['slice'] >= 600]  # only use training set

    label = [0]*cluster_num

    for i in range(cluster_num):
        one_cluster = df.loc[df['current_cluster'] == i]
        stats = one_cluster.mean()
        precisions = [stats[2], stats[4], stats[6], stats[8]]
        fscores = [stats[3], stats[5], stats[7], stats[9]]
        p_max = max(precisions)
        p_idx = np.argmax(precisions)
        f_idx = np.argmax(fscores)
        if mode in [1,2]:
            if p_max <= threshold:
                idx = f_idx
            else:
                idx = p_idx
        elif mode == 3:
            idx = f_idx
        else:
            idx = p_idx
            
        class_num = idx + 1
        label[i] = class_num

    return label

In [12]:
from auto_label import PRECISION_SOLO

In [28]:
seg_model = 'k-means'
seg_nd = '4d'
cluster_num = 128
mode = PRECISION_SOLO
threshold = 0

In [29]:
label = auto_label_last_201(seg_model, seg_nd, cluster_num, mode, threshold)

In [15]:
len(label)

128

In [30]:
img = color_unsupervised_pred_by_slice(label, seg_model, seg_nd, cluster_num, slice_num=471)

In [31]:
cv2.imwrite('report_images\k-means_4d_128_report_471.png', img)

True

#### Transfer tif to png

In [4]:
p1 = os.path.join(os.getcwd(), 'VA10_Pc200_Ram25_Pf50_T125_0140\VA10_Pc200_Ram25_Pf50_T125_0140_0001.rec.8bit.tif')
img1 = cv2.imread(p1, -1)

cv2.imwrite('report_images/raw_data_to_png/VA10_Pc200_Ram25_Pf50_T125_0140_0001.rec.8bit.png', img1)

True

### plots for small clusters performance 

In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import csv
import glob
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support

In [2]:
# dragonfly
def get_dragonfly_label_by_slice(slice_num):
    current_path = os.getcwd()
    s = slice_num - 400 + 1
    s = str(s).zfill(3)
    dragonfly_bassanite = os.path.join(current_path, 'VA10_0050_Bassanite\VA10_0050_Bassanite{}.tiff'.format(s))
    dragonfly_celestite = os.path.join(current_path, 'VA10_0050_Celestite\VA10_0050_Celestite{}.tiff'.format(s))
    dragonfly_gypsum = os.path.join(current_path, 'VA10_0050_Gypsum\VA10_0050_Gypsum{}.tiff'.format(s))
    dragonfly_pore = os.path.join(current_path, 'VA10_0050_Pores\VA10_0050_Pores{}.tiff'.format(s))

    bassanite = cv2.imread(dragonfly_bassanite)
    celestite = cv2.imread(dragonfly_celestite)
    gypsum = cv2.imread(dragonfly_gypsum)
    pore = cv2.imread(dragonfly_pore)

    # transfer the reference into 0/1 labels and then turn to its class number
    # 0: other;
    # 1: pore;
    # 2: gypsum;
    # 3: celestite;
    # 4: bassanite
    pore_label = (pore == [255,255,255]).all(axis=2) * 1
    gypsum_label = (gypsum == [255,255,255]).all(axis=2) * 2
    celestite_label = (celestite == [255,255,255]).all(axis=2) * 3
    bassanite_label = (bassanite == [255,255,255]).all(axis=2) * 4

    label = pore_label + gypsum_label + celestite_label + bassanite_label

    return label.flatten()

In [3]:
# gmm / k-means
def get_unsupervised_pred_by_slice(manual_label, seg_model, seg_nd, cluster_num, slice_num):
    current_path = os.getcwd()
    seg_path = os.path.join(current_path, 'new_large_clusters_rec', seg_model, seg_nd, 'cluster_{}'.format(cluster_num))
    seg_res_list = glob.glob(os.path.join(seg_path, str(slice_num), '*.png'))
    
    img = 0

    assert len(manual_label) == len(seg_res_list) == cluster_num

    for seg in seg_res_list:
        # our segmentation results has invered color, black [0,0,0] is the interested class
        c = int(os.path.basename(seg[-16:-13]))
        if manual_label[c] == 1:  # pore
            pore = cv2.imread(seg)
            pore = (pore == [0,0,0]).all(axis=2) * 1
            img += pore
        elif manual_label[c] == 2:  #gypsum
            gypsum = cv2.imread(seg)
            gypsum = (gypsum == [0,0,0]).all(axis=2) * 2
            img += gypsum
        elif manual_label[c] == 3:  #celestite
            celestite = cv2.imread(seg)
            celestite = (celestite == [0,0,0]).all(axis=2) * 3
            img += celestite
        elif manual_label[c] == 4:  #bassanite
            bassanite = cv2.imread(seg)
            bassanite = (bassanite == [0,0,0]).all(axis=2) * 4
            img += bassanite

    return img.flatten()

In [4]:
def evaluate_label_per_slice(seg_model, seg_nd, cluster_num, slice_num, class_label):
    # get class labels assigned to each cluster
    
    # get pixel label and pred of the given slice
    pixel_pred = get_unsupervised_pred_by_slice(class_label, seg_model, seg_nd, cluster_num, slice_num)
    pixel_label = get_dragonfly_label_by_slice(slice_num)

    p, r, f1, _= precision_recall_fscore_support(pixel_label, pixel_pred, average=None, labels=[1,2,3,4])
    macro_f1 = f1.mean()

    return p, r, f1, macro_f1

In [19]:
model = 'gmm'
nd = '4d'
c = 4


In [20]:
header = ['slice', 'pore_precision', 'pore_recall', 'pore_f1', 'gypsum_precision', 'gypsum_recall', 'gypsum_f1', 
          'celestite_precision', 'celestite_recall', 'celestite_f1', 'bassanite_precision', 'bassanite_recall', 'bassanite_f1', 'macro_f1_4', 'macro_f1_3']
f = open('{}_{}_{}_label.csv'.format(model, nd, c), 'w')
writer = csv.writer(f)
writer.writerow(header)

class_label = [2, 2, 1, 2]
for s in range(400, 500):
    p, r, f1, macro_f1_4 = evaluate_label_per_slice(model, nd, c, s, class_label)
    f1_3 = np.array([f1[0], f1[1], f1[3]])
    macro_f1_3 = f1_3.mean()
    data = [s, p[0], r[0], f1[0], p[1], r[1], f1[1], p[2], r[2], f1[2], p[3], r[3], f1[3], macro_f1_4, macro_f1_3]
    writer.writerow(data)

f.close()
print('Finish!')

c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavi

Finish!


c:\Users\s1923493\AppData\Local\miniconda3\envs\test-uninstall\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
kmeans_3d = pd.read_csv('k-means_3d_4_label.csv')
kmeans_4d = pd.read_csv('k-means_4d_4_label.csv')

In [11]:
kmeans_3d.describe()

,slice,pore_precision,pore_recall,pore_f1,gypsum_precision,gypsum_recall,gypsum_f1,celestite_precision,celestite_recall,celestite_f1,bassanite_precision,bassanite_recall,bassanite_f1,macro_f1_4,macro_f1_3
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,100.0,100.0,100.000000,100.000000,100.000000,100.000000,100.000000
mean,449.500000,0.478251,0.746430,0.582869,0.817376,0.951128,0.879131,0.0,0.0,0.0,0.924505,0.310458,0.464420,0.481605,0.642140
std,29.011492,0.011706,0.006983,0.008395,0.014171,0.003237,0.008510,0.0,0.0,0.0,0.056613,0.006641,0.009581,0.002994,0.003992
min,400.000000,0.457693,0.725799,0.567521,0.797498,0.943686,0.866640,0.0,0.0,0.0,0.836373,0.293389,0.443855,0.474334,0.632445
25%,424.750000,0.470457,0.743152,0.576872,0.805211,0.949597,0.872036,0.0,0.0,0.0,0.868816,0.305343,0.459008,0.479726,0.639635
50%,449.500000,0.476812,0.747461,0.582375,0.811318,0.952177,0.875776,0.0,0.0,0.0,0.930472,0.311751,0.464119,0.481834,0.642445
75%,474.250000,0.485731,0.751438,0.588147,0.833323,0.953640,0.888751,0.0,0.0,0.0,0.983210,0.315111,0.472118,0.483525,0.644700
max,499.000000,0.511346,0.760538,0.606694,0.840080,0.956146,0.892558,0.0,0.0,0.0,0.992493,0.326675,0.491341,0.488038,0.650717


In [12]:
kmeans_4d.describe()

,slice,pore_precision,pore_recall,pore_f1,gypsum_precision,gypsum_recall,gypsum_f1,celestite_precision,celestite_recall,celestite_f1,bassanite_precision,bassanite_recall,bassanite_f1,macro_f1_4,macro_f1_3
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,100.0,100.0,100.000000,100.000000,100.000000,100.000000,100.000000
mean,449.500000,0.119283,0.149143,0.132525,0.832855,0.790674,0.811206,0.0,0.0,0.0,0.385627,0.501088,0.435535,0.344817,0.459755
std,29.011492,0.008925,0.010716,0.009531,0.014029,0.009839,0.011620,0.0,0.0,0.0,0.021030,0.010213,0.014951,0.003203,0.004271
min,400.000000,0.098518,0.123800,0.109722,0.814181,0.773654,0.794849,0.0,0.0,0.0,0.340540,0.479642,0.399008,0.337011,0.449349
25%,424.750000,0.113532,0.142677,0.127221,0.820284,0.782660,0.801036,0.0,0.0,0.0,0.365432,0.492952,0.426215,0.342809,0.457079
50%,449.500000,0.119802,0.148993,0.132397,0.827294,0.788210,0.806344,0.0,0.0,0.0,0.388561,0.502741,0.435517,0.345366,0.460488
75%,474.250000,0.126500,0.155930,0.139019,0.848956,0.800841,0.824449,0.0,0.0,0.0,0.403460,0.508454,0.447323,0.347275,0.463034
max,499.000000,0.135812,0.173223,0.152185,0.853495,0.807492,0.829262,0.0,0.0,0.0,0.419314,0.525690,0.465481,0.350371,0.467161


In [21]:
gmm_3d = pd.read_csv('gmm_3d_4_label.csv')
gmm_4d = pd.read_csv('gmm_4d_4_label.csv')

In [22]:
gmm_3d.describe()

,slice,pore_precision,pore_recall,pore_f1,gypsum_precision,gypsum_recall,gypsum_f1,celestite_precision,celestite_recall,celestite_f1,bassanite_precision,bassanite_recall,bassanite_f1,macro_f1_4,macro_f1_3
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,100.0,100.0,100.000000,100.000000,100.000000,100.000000,100.000000
mean,449.500000,0.336632,0.861083,0.483846,0.951403,0.816908,0.878836,0.0,0.0,0.0,0.456412,0.568664,0.506271,0.467239,0.622985
std,29.011492,0.010373,0.015097,0.009414,0.005432,0.029627,0.019402,0.0,0.0,0.0,0.021756,0.013242,0.018197,0.010388,0.013851
min,400.000000,0.311206,0.830025,0.460213,0.943842,0.770080,0.848399,0.0,0.0,0.0,0.382769,0.547260,0.451836,0.440858,0.587810
25%,424.750000,0.329558,0.850799,0.477714,0.946195,0.790756,0.861079,0.0,0.0,0.0,0.446264,0.557345,0.495282,0.459566,0.612755
50%,449.500000,0.336084,0.857536,0.484200,0.950114,0.809149,0.874057,0.0,0.0,0.0,0.462642,0.567590,0.511100,0.470726,0.627635
75%,474.250000,0.343566,0.877426,0.489017,0.957313,0.847056,0.898789,0.0,0.0,0.0,0.469906,0.577711,0.517538,0.475581,0.634108
max,499.000000,0.358189,0.888624,0.505194,0.961033,0.860904,0.908112,0.0,0.0,0.0,0.497121,0.595360,0.540102,0.481077,0.641437


In [23]:
gmm_4d.describe()

,slice,pore_precision,pore_recall,pore_f1,gypsum_precision,gypsum_recall,gypsum_f1,celestite_precision,celestite_recall,celestite_f1,bassanite_precision,bassanite_recall,bassanite_f1,macro_f1_4,macro_f1_3
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,100.0,100.0,100.0,100.0,100.0,100.000000,100.000000
mean,449.500000,0.131186,0.477497,0.205611,0.811807,0.877602,0.843394,0.0,0.0,0.0,0.0,0.0,0.0,0.262251,0.349668
std,29.011492,0.019147,0.038355,0.026931,0.017694,0.025105,0.020792,0.0,0.0,0.0,0.0,0.0,0.0,0.011877,0.015837
min,400.000000,0.100069,0.396856,0.163044,0.789245,0.830260,0.811726,0.0,0.0,0.0,0.0,0.0,0.0,0.243793,0.325057
25%,424.750000,0.117249,0.449597,0.185931,0.796819,0.859237,0.823290,0.0,0.0,0.0,0.0,0.0,0.0,0.252779,0.337039
50%,449.500000,0.128153,0.464027,0.202476,0.803161,0.872627,0.835718,0.0,0.0,0.0,0.0,0.0,0.0,0.259552,0.346070
75%,474.250000,0.151321,0.520597,0.234637,0.833668,0.901952,0.867517,0.0,0.0,0.0,0.0,0.0,0.0,0.275084,0.366778
max,499.000000,0.162302,0.548330,0.249346,0.840025,0.913253,0.874353,0.0,0.0,0.0,0.0,0.0,0.0,0.280201,0.373602


In [3]:
vgg = pd.read_csv('test_k-means_3d_128.csv')
vgg

,slice,pore_precision,pore_recall,pore_f1,gypsum_precision,gypsum_recall,gypsum_f1,celestite_precision,celestite_recall,celestite_f1,bassanite_precision,bassanite_recall,bassanite_f1,macro_p,macro_r,macro_f
0,400,0.670188,0.133343,0.222431,0.847251,0.979397,0.908544,0.000000,0.000000,0.000000,0.705533,0.598460,0.647601,0.555743,0.427800,0.444644
1,401,0.707541,0.106458,0.185069,0.843109,0.979294,0.906113,0.000000,0.000000,0.000000,0.699244,0.569473,0.627721,0.562474,0.413806,0.429726
2,402,0.757836,0.091350,0.163046,0.839535,0.976778,0.902971,0.000000,0.000000,0.000000,0.679983,0.544313,0.604631,0.569338,0.403110,0.417662
3,403,0.735008,0.112623,0.195318,0.836372,0.984139,0.904258,0.000000,0.000000,0.000000,0.712995,0.525798,0.605253,0.571094,0.405640,0.426207
4,404,0.649459,0.130933,0.217930,0.844590,0.977469,0.906184,0.000000,0.000000,0.000000,0.696995,0.551932,0.616039,0.547761,0.415084,0.435038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,495,0.548483,0.243043,0.336830,0.886568,0.978921,0.930458,0.964178,0.871091,0.915274,0.704925,0.620200,0.659854,0.776039,0.678314,0.710604
96,496,0.626277,0.324935,0.427873,0.888072,0.978061,0.930897,0.968740,0.860370,0.911345,0.716258,0.620940,0.665202,0.799837,0.696077,0.733829
97,497,0.587918,0.277678,0.377201,0.890060,0.973089,0.929724,0.962594,0.859928,0.908369,0.688850,0.630082,0.658157,0.782355,0.685194,0.718363
98,498,0.542260,0.279144,0.368561,0.893267,0.969073,0.929627,0.967793,0.842354,0.900727,0.648385,0.612614,0.629992,0.762926,0.675796,0.707227


In [4]:
vgg.describe()

,slice,pore_precision,pore_recall,pore_f1,gypsum_precision,gypsum_recall,gypsum_f1,celestite_precision,celestite_recall,celestite_f1,bassanite_precision,bassanite_recall,bassanite_f1,macro_p,macro_r,macro_f
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,449.500000,0.679273,0.222682,0.330162,0.860189,0.978078,0.915231,0.739044,0.609446,0.659643,0.717102,0.587355,0.644452,0.748902,0.599390,0.637372
std,29.011492,0.076717,0.054697,0.061084,0.018679,0.002961,0.009928,0.395577,0.360171,0.373835,0.032081,0.036415,0.019211,0.099576,0.099018,0.102624
min,400.000000,0.482291,0.091350,0.163046,0.834750,0.968614,0.901249,0.000000,0.000000,0.000000,0.633114,0.525038,0.589561,0.545566,0.403110,0.417662
25%,424.750000,0.627650,0.184481,0.290209,0.845490,0.976655,0.907107,0.864286,0.304116,0.452637,0.698384,0.558102,0.629928,0.762444,0.523182,0.588910
50%,449.500000,0.685725,0.215296,0.329824,0.853227,0.978613,0.910806,0.954531,0.837595,0.894188,0.713032,0.581181,0.644572,0.795966,0.651758,0.689367
75%,474.250000,0.729730,0.254967,0.368645,0.881822,0.980085,0.927123,0.962924,0.862463,0.909047,0.726471,0.621244,0.659441,0.807553,0.669992,0.706686
max,499.000000,0.894704,0.358377,0.472448,0.896064,0.984139,0.931610,0.974510,0.887928,0.926375,0.809326,0.654784,0.676845,0.855516,0.706957,0.738655
